In [6]:
# Road Trip Planner

import collections
import csv
import math
import random

FILENAME = 'usa_cities.csv'
# FILENAME = 'california.csv'
START_POINT = 'San Francisco, California'
END_POINT = 'Dallas, Texas'
# END_POINT = 'San Diego, California'
# END_POINT = 'Santa Cruz, California'
KM_PER_DAY = 250
TRAVEL_MODE = 'ADVENTURE'  # pick from SHORTEST or ADVENTURE
BORING_FACTOR = 0.9  # 1 not boring, 0 super boring

def parse_dataset(reader):
    """Parses the raw CSV data into a dictionary of City namedtuples"""
    
    # Define namedtuple to store city information
    City = collections.namedtuple('City', ['name', 'lat', 'lon', 'county', 'state'])

    database = {}
    for idx, line in enumerate(reader):
        # example line
        # city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
        # "California","California","40.0692","-79.9152","United States","US","USA","Pennsylvania","","6364","1840003642"
        try:
            name, county, state, lat, lon = line
            lat, lon = degrees_to_radians(lat), degrees_to_radians(lon)
        except:
            print(f'Could not parse file at line {idx}: {line}')
            continue
        else:
            city = City(name, lat, lon, county, state)
            database[(name, state)] = city

    if len(database) < 1:
        raise Exception('Could not parse *any* data from input file')

    return database


def retrieve_city_from_db(city_string, database):
    """Searches the database for a matching City
    
    'city_string' must be in the form "name, state".
    """
        
    name, state = map(str.strip, city_string.split(','))
    try:
        city = database[(name, state)]
    except KeyError:
        raise Exception(f'City not found in database: {name}, {state}')
    else:
        print(f'Found match for {name}, {state}: {city}')
        return city


def degrees_to_radians(value):
    """Converts a lat/lon value in degrees to radians"""
    return float(value) * 3.14 / 180


def haversine(lat_i, lon_i, lat_j, lon_j):
    """Returns the distance between two points using the haversine formula"""
   
    # Haversine formula for distances between points on a sphere
    # https://en.wikipedia.org/wiki/Haversine_formula
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i
    sinsq_dlathalf = math.sin(dlat/2)
    sinsq_dlonhalf = math.sin(dlon/2)
    
    a = (sinsq_dlathalf*sinsq_dlathalf) + math.cos(lat_i) * math.cos(lat_j) * (sinsq_dlonhalf*sinsq_dlonhalf)
    c = 2 * math.atan2( math.sqrt(a), math.sqrt(1-a) )
    d = 6373  * c  # R is 'a' radius of earth
    
    return d


def get_city_neighbors(query, database, radius=500):
    """Returns neighbors of a given city"""
    
    return (city for city in database.values() 
            if haversine(city.lat, city.lon, query.lat, query.lon) <= radius)

#     for city in database.values():
#         if haversine(city.lat, city.lon, query.lat, query.lon) <= radius:
#             yield city


def take_shortest_route(neighbors, route, end_point):
    """Picks the next stop in the route by calculating distances
    between cities and the end point and choosing the shortest"""
    
    d_to_end = lambda c: haversine(c.lat, c.lon, end_point.lat, end_point.lon)
    
    sorted_neighbors = sorted(neighbors, key=d_to_end)
    for city in sorted_neighbors:
        if city not in route:
            break
    else:
        raise Exception('Route is not possible with current parameters')
        
    return city  


def take_adventure_route(neighbors, route, end_point, boringness_factor=BORING_FACTOR):
    """Picks the next stop somewhat randomly"""
    
    if random.random() >= boringness_factor:
        return take_shortest_route(neighbors, route, end_point)
    
    neighbors = list(neighbors)  # generator to list
    random.shuffle(neighbors)
    for city in neighbors:
        if city not in route:
            return city

    raise Exception('Adventure mode failed. Try a more boring route.')


# Define search modes
picker = {'SHORTEST': take_shortest_route,
          'ADVENTURE': take_adventure_route}
#
## Main Code
#

pick_next_city = picker.get(TRAVEL_MODE)
if pick_next_city is None:
    raise Exception("Travel mode '{TRAVEL_MODE}' not supported.")

with open(FILENAME, encoding='utf-8') as handle:
    
    reader = csv.reader(handle)
    next(reader)  # skip first line, header
    
    database = parse_dataset(reader)

print(f'Read {len(database)} cities from database')

sp_city = retrieve_city_from_db(START_POINT, database)
ep_city = retrieve_city_from_db(END_POINT, database)

# Find 'best' route
route = collections.OrderedDict()
route = route.fromkeys([sp_city])
    
current_city = sp_city
while True:
    neighbors = get_city_neighbors(current_city, database, radius=KM_PER_DAY)
    current_city = pick_next_city(neighbors, route, ep_city)
    route[current_city] = None
    if current_city == ep_city:
        break

print(f'Found route with {len(route)} stops')
for idx, city in enumerate(route, start=1):
    print(f'City #{idx}: {city.name}, {city.state}')

Read 29737 cities from database
Found match for San Francisco, California: City(name='San Francisco', lat=0.656293912, lon=-2.1348850231111114, county='San Mateo', state='California')
Found match for Dallas, Texas: City(name='Dallas', lat=0.5720109914444444, lon=-1.6886988033333334, county='Dallas', state='Texas')
Found route with 81 stops
City #1: San Francisco, California
City #2: Le Grand, California
City #3: Kettleman City, California
City #4: Springville, California
City #5: Death Valley, California
City #6: Red Mountain, California
City #7: Cal Nev Ari, Nevada
City #8: Sloan, Nevada
City #9: Enterprise, Utah
City #10: Elsinore, Utah
City #11: Toquerville, Utah
City #12: Cameron, Arizona
City #13: Sun Valley, Arizona
City #14: Tucson, Arizona
City #15: Gila Bend, Arizona
City #16: Camp Verde, Arizona
City #17: Pinetop, Arizona
City #18: Pie Town, New Mexico
City #19: Buckhorn, New Mexico
City #20: Arrey, New Mexico
City #21: Clifton, Arizona
City #22: Mesilla, New Mexico
City #23:

In [10]:
import simplekml

def radians_to_degrees(value):
    """Converts a lat/lon value in radians to degrees"""
    return float(value) * 180 / 3.14

kml = simplekml.Kml()

for city in route:
    kml.newpoint(name=city.name, coords=[(radians_to_degrees(city.lon), radians_to_degrees(city.lat))])
    
kml.save('route2.kml')